In [1]:
import pandas as pd

for coin in ['BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'DOGEUSDT']:
    cols = [
        "open_time", "open", "high", "low", "close", "volume",
        "close_time", "quote_volume", "num_trades",
        "tbb_volume", "tbq_volume", "ignore"
    ]

    df = pd.read_csv(f"data/{coin}.csv", header=None, names=cols)

    # Convert to proper dtypes
    df["open_time"] = df["open_time"].astype("int64")
    df["close_time"] = df["close_time"].astype("int64")

    numeric_cols = [
        "open", "high", "low", "close", "volume",
        "quote_volume", "num_trades", "tbb_volume", "tbq_volume"
    ]
    df[numeric_cols] = df[numeric_cols].astype(float)

    # ---- 2. Use open_time (ms) as datetime index for resampling ----
    df["dt"] = pd.to_datetime(df["open_time"], unit="ms")
    df = df.set_index("dt")

    # ---- 3. Resample to hourly candles ----
    # Rules:
    #   open  = first open of the hour
    #   high  = max high
    #   low   = min low
    #   close = last close
    #   volume, quote_volume, trades, taker buy volumes = sum
    #   ignore = last (usually 0 anyway)
    agg_dict = {
        "open": "first",
        "high": "max",
        "low": "min",
        "close": "last",
        "volume": "sum",
        "quote_volume": "sum",
        "num_trades": "sum",
        "tbb_volume": "sum",
        "tbq_volume": "sum",
        "ignore": "last",
    }

    hourly = df.resample("D").agg(agg_dict)

    # Drop empty hours (no trades)
    hourly = hourly.dropna(subset=["open", "high", "low", "close"])

    # ---- 4. Rebuild open_time and close_time in ms (Binance style) ----
    # Index is start-of-hour; convert to ms
    hourly["open_time"] = (hourly.index.view("int64") // 10**6).astype("int64")

    # close_time = end of the hour minus 1 ms
    ONE_HOUR_MS = 60 * 60 * 1000
    hourly["close_time"] = hourly["open_time"] + ONE_HOUR_MS - 1

    # ---- 5. Reorder columns to match original exactly ----
    hourly_out = hourly[
        [
            "open_time",
            "open",
            "high",
            "low",
            "close",
            "volume",
            "close_time",
            "quote_volume",
            "num_trades",
            "tbb_volume",
            "tbq_volume",
            "ignore",
        ]
    ]

    # ---- 6. Write back out: same style (no header, comma-separated) ----
    hourly_out.to_csv(
        f"{coin}.csv",
        header=False,
        index=False,
        float_format="%.8f"  # adjust precision if you like
    )
